In [2]:
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.metrics import r2_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from python_module.pricing_model import BSMModel, SABRModel
from python_module.weights_optimization import find_min_abs_weights, fast_find_min_abs_weights

In [3]:
# Market Data Generation Inputs
F = 100
T = 1
alpha = 0.2
beta = 1
rho = -0.9
nu = 0.3
r = 0

In [4]:
# Generate options
results = dict()
for K in range(60, 141, 1):
    option_type = 'call' if K > F else 'put'
    out = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type=option_type)
    out['option_type'] = option_type
    out['delta_call'] = SABRModel.compute_option(F, K, T, alpha, beta, rho, nu, r, option_type='call')['delta']
    results[K] = out

In [5]:
# Generate Weights
df = pd.DataFrame(results)
df = df.transpose()
df['weight'] = (1 / np.power(df.index, 2)) * 1_000_000

In [6]:
# Compute COG
X = df[['gamma', 'vanna', 'volga']]
y = df['theta']
model = LinearRegression(fit_intercept=False)
model.fit(X.values, y.values)
y_pred = model.predict(X.values)
coef = model.coef_
r2 = r2_score(y.values, y_pred)
cogwa = (X * (coef))
cogwa.columns = ['costof_gamma', 'costof_vanna', 'costof_volga']
df = pd.concat([df, cogwa], axis=1)
df['costof_other'] = df['theta'] - df[['costof_gamma', 'costof_vanna', 'costof_volga']].sum(axis=1)

In [7]:
# Delta Bucketing
bins = [0, 0.25, 0.5, 0.75, 1]
labels = ['0-0.25', '0.25-0.5', '0.5-0.75', '0.75-1']
df['delta_bucket'] = pd.cut(df['delta_call'], bins=bins, labels=labels)

In [8]:
# Compute adjusted ref risk for portfolio reduction
risk_ref = 'theta'
df[f'pos_{risk_ref}'] = df[risk_ref] * df['weight']

In [9]:
# Compute weight_reduction with discrete approach
df['risk_bucket'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)[f'pos_{risk_ref}'].sum().to_dict()))
df['qty_to_trade'] = df['risk_bucket'] / df[risk_ref]
df['min_qty_to_trade'] = list(df['delta_bucket'].map(df.groupby('delta_bucket', observed=True)['qty_to_trade'].min().to_dict())) 
df['weight_reduction'] = (df['qty_to_trade']==df['min_qty_to_trade']) * df['min_qty_to_trade']
df['weight_reduction'] = df['weight_reduction'].replace(0, np.nan)

In [10]:
# Compute weight_reduction with optimized approach
index_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga', 'costof_other']].multiply(df['weight'], axis=0).sum()
weights = fast_find_min_abs_weights(df, index_cog, combination_size=4)
df.loc[weights.index, 'weight_opt'] = weights.values
opt_cog = df[['costof_gamma', 'costof_vanna', 'costof_volga', 'costof_other']].multiply(df['weight_opt'], axis=0).sum()

In [11]:
px.scatter(df[['weight', 'weight_reduction', 'weight_opt']])

In [12]:
df['weight_reduction']

60    NaN
61    NaN
62    NaN
63    NaN
64    NaN
       ..
136   NaN
137   NaN
138   NaN
139   NaN
140   NaN
Name: weight_reduction, Length: 81, dtype: float64